In [31]:
import pandas as pd
import numpy as np
import reduction_function as rfun
import polarization_function as pfun
import networkx as nx

In [32]:
net_file = pd.read_csv("output_network/network_filtered.csv", sep="\t")
G = nx.from_pandas_edgelist(net_file, "src", "trg")
print(G)

Graph with 511 nodes and 2785 edges


#### In the real scenario, we won't be reading network_users_combined.csv, we will be getting the reclustered labelled monthly messages on each topic and merging them in single df. 

#### After that we will have to apply averaging for the users who write more than 1 message for a single topic.

In [33]:
data = pd.read_csv("output_network/network_users_combined.csv", sep="\t")
data.head(), data.shape

(            index  nodeid
 0     novagenesis     473
 1         jscoppe    6978
 2           sje46    8280
 3  reconditecache    8339
 4         Delheru   14848,
 (511, 2))

In [34]:
#dataset for the other 2 topics
data2 = data.copy()
data3 = data.copy()

Each dataset holds the person's stance on a single topic

In [35]:
def simulate_data(data, topic):
    n_rows = data.shape[0]
    data["A"] = np.random.rand(n_rows)
    data["F"] = np.random.rand(n_rows)
    data["N"] = np.random.rand(n_rows)
    data[f"stance_{topic}"] = data.apply(lambda x: rfun.get_stance(x), axis = 1)
    return data

In [36]:
data1 = simulate_data(data, "lib")
data2 = simulate_data(data2, "abortion")
data3 = simulate_data(data3, "marriage")

In [39]:
data3.head()

,index,nodeid,A,F,N,stance_marriage
0,novagenesis,473,0.252707,0.881912,0.387267,-0.881912
1,jscoppe,6978,0.790447,0.675034,0.808380,0.790447
2,sje46,8280,0.161014,0.294906,0.401649,-0.294906
3,reconditecache,8339,0.343747,0.319886,0.251532,0.343747
4,Delheru,14848,0.658554,0.672088,0.205418,-0.672088


UMAP and PCA on the simulated data

In [42]:
# full_data = rfun.merge_datasets(data1, data2, data3) ##use this on the real data
merged_df = pd.merge(pd.merge(data1, data2, on=['nodeid', 'index']), data3, on=['nodeid', 'index'])

In [43]:
merged_df.head()

,index,nodeid,A_x,F_x,N_x,stance_lib,A_y,F_y,N_y,stance_abortion,A,F,N,stance_marriage
0,novagenesis,473,0.890809,0.747099,0.533479,0.890809,0.704796,0.557835,0.707656,0.704796,0.252707,0.881912,0.387267,-0.881912
1,jscoppe,6978,0.332314,0.773613,0.059779,-0.773613,0.102991,0.275417,0.935592,-0.275417,0.790447,0.675034,0.808380,0.790447
2,sje46,8280,0.566678,0.637696,0.026786,-0.637696,0.234849,0.308586,0.730204,-0.308586,0.161014,0.294906,0.401649,-0.294906
3,reconditecache,8339,0.248231,0.359356,0.043993,-0.359356,0.223438,0.559271,0.050597,-0.559271,0.343747,0.319886,0.251532,0.343747
4,Delheru,14848,0.971852,0.410611,0.268629,0.971852,0.399841,0.269053,0.109526,0.399841,0.658554,0.672088,0.205418,-0.672088
